Problema tomado de: https://github.com/fferegrino/cf-ml/blob/main/car-prices/car-price.ipynb

# Predicción de automóviles usados

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
from pandas_profiling import ProfileReport
import os
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
cars = pd.read_csv('cars.csv')

In [3]:
cars.head()

,maker,model,year,transmission,mileage,fuelType,tax,mpg,engineSize,price
0,cclass,C Class,2020,Automatic,1200,Diesel,NaN,NaN,2.0,30495
1,cclass,C Class,2020,Automatic,1000,Petrol,NaN,NaN,1.5,29989
2,cclass,C Class,2020,Automatic,500,Diesel,NaN,NaN,2.0,37899
3,cclass,C Class,2019,Automatic,5000,Diesel,NaN,NaN,2.0,30399
4,cclass,C Class,2019,Automatic,4500,Diesel,NaN,NaN,2.0,29899


## Análisis Exploratorio de Datos

In [4]:
profile = ProfileReport(cars, title="Raw Car Dataset Analysis", explorative=True)
profile.to_file("cars-report.html")

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 99.36it/s]


In [5]:
print(os.getcwd() + "/cars-report.html")

d:\Notebooks\practico_ML/cars-report.html
